# Interacting equivariant objects: Linear, TensorProducts, S2Grid, Equivariant Nonlinearities

So now that we have features in terms of irreps, what can we do with them? In order to preserve equivariance, all operations we use must be equivariant.

In [1]:
import e3nn
from e3nn import o3

In [2]:
o3.TensorProduct

e3nn.o3.tensor_product.TensorProduct

In [3]:
o3.Linear

e3nn.o3.linear.Linear

### All learnable parameters must be scalars

In order to preserve equivariance, all learnable parameters in the model must be scalars. For $f: X, W \rightarrow Y$,

$f(D_X(g)x, w) = D_Y(g)f(x, w), \text{ if } \forall g \in G, D_W(g) = I$

For the irreps of $O(3)$, this condition is only met if $w$ is purely scalar.

Otherwise, we would need to rotate our learnable features with our input data, which largely defeats the purpose of creating an equivariant neural network.